<a href="https://colab.research.google.com/github/sizhaoliu/StitchDataConnectors/blob/master/StitchDataSources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# importing the libraries
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd 

In [0]:
# Step 1: Sending a HTTP request to a URL
url = "https://www.stitchdata.com/integrations/sources/"
# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text


In [0]:
# Step 2: Parse the html content
soup = BeautifulSoup(html_content, "lxml")
# print(soup.prettify()) # print the parsed data of html


In [4]:
# Step 3: Analyze the HTML tag, where your content lives
# Get the section having the id "sources"
data_source_section = soup.find("section", attrs={"id": "sources"})

data_source_list = data_source_section.ul.find_all("li", attrs={"style": ""}) 
# 3 elements with the following IDs are excluded: requestBox, analysisToolBox, destinationBox

# print(data_source_list[0].prettify()) # print the first data source element

print(str(len(data_source_list)) + " sources in total.")

123 sources in total.


In [5]:
data_source_table = []
for li in data_source_list: # loop on all li's
    t_row = {}
    # Each table row is stored in the form of
    # t_row = {'Connector Name': '', 'Category': '', 'Paid Status': '', 'Support Status': ''}

    t_row['Connector Name'] = li.a.div.p.text
    t_row['Category'] = li.attrs['data-integration-category-name']

    status_div = li.a.find("div", attrs={"class": "logo-pills"})
    t_row['Paid Status'] = status_div.find("div", attrs={"class": "paid-status"}).text.strip()
    t_row['Support Status'] = status_div.find("div", attrs={"class": "support-status"}).text.strip()
    data_source_table.append(t_row)

print(data_source_table[0])

{'Connector Name': 'AdRoll', 'Category': 'Advertising', 'Paid Status': 'Free', 'Support Status': 'Stitch-certified'}


In [6]:
# Step 4: Export the data to csv
with open("StitchDataSourceList.csv", 'w') as out_file:
      # The table has headers as following
      headers = [ 
          "Connector Name",
          "Category",
          "Paid Status",
          "Support Status"
      ]
      writer = csv.DictWriter(out_file, headers)
      # write the header
      writer.writeheader()
      for row in data_source_table:
          if row:
              writer.writerow(row)

data = pd.read_csv("StitchDataSourceList.csv") 
data.head(200)

,Connector Name,Category,Paid Status,Support Status
0,AdRoll,Advertising,Free,Stitch-certified
1,AfterShip,Business,Free,Stitch-certified
2,Amazon Aurora MySQL,Databases,Free,Stitch-certified
3,Amazon Aurora PostgreSQL,Databases,Free,Stitch-certified
4,Amazon RDS for MariaDB,Databases,Free,Stitch-certified
...,...,...,...,...
118,Yotpo,Business,Free,Community-supported
119,Zapier,Automation,Free,Stitch-certified
120,Zendesk Chat,Support,Free,Community-supported
121,Zendesk Support,Support,Standard,Stitch-certified
